<a href="https://colab.research.google.com/github/CivilKen/Image-Sentiment-Classification/blob/master/VGGnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
!ls "/content/gdrive/My Drive/ML_homework/Colab Notebooks/hw3"

Mounted at /content/gdrive
 df_test_feature   'hw3_v3.ipnb '	      nor_df_valid_feature
 df_train_feature   hw3_v4.ipnb		      ParseData.ipynb
 df_train_label    'hw3_v4.ipnb 的副本 (1)'   StoredModel
 df_valid_feature   hw3_v4_restore.ipnb       test.csv
 df_valid_label     hw3_v5.ipnb		      test_feature.csv
 hw3_v1.ipynb	    nor_df_test_feature       train.csv
 hw3_v2.ipynb	    nor_df_train_feature      train_feature.csv


In [0]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from skimage import io 
import matplotlib.pyplot as plt
path = "/content/gdrive/My Drive/ML_homework/Colab Notebooks/hw3/"

In [0]:
#test GPU
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
#read dataframe 
df_train_feature = pd.read_pickle(path+"df_train_feature")
df_valid_feature = pd.read_pickle(path+"df_valid_feature")
df_test_feature = pd.read_pickle(path+"df_test_feature")
df_train_label = pd.read_pickle(path+"df_train_label")
df_valid_label = pd.read_pickle(path+"df_valid_label")
nor_df_train_feature = pd.read_pickle(path+"nor_df_train_feature")
nor_df_valid_feature = pd.read_pickle(path+"nor_df_valid_feature")
nor_df_test_feature = pd.read_pickle(path+"nor_df_test_feature")

In [0]:
# parse label data
def one_hot(df_label,lenth):
  #original input is dataframe
  array_label = np.zeros([lenth,7])
  for i in range(lenth):
    idx = df_label.loc[i]
    array_label[i, idx]=1
  return array_label

arr_train_label = one_hot(df_train_label, df_train_label.shape[0])
arr_valid_label = one_hot(df_valid_label, df_valid_label.shape[0])

In [0]:
'''
# define accuracy
def accuracy(labels, pred):
  #both labels and pred are in the shape of one-hot encoding
  #both are in the format of array
  accu=0
  for i in range(len(labels)):
    lab_val = np.argmax(labels[i,:])
    pred_val = np.argmax(pred[i,:])
    if lab_val==pred_val:
      accu+=1
  return accu/len(labels)
'''

'\n# define accuracy\ndef accuracy(labels, pred):\n  #both labels and pred are in the shape of one-hot encoding\n  #both are in the format of array\n  accu=0\n  for i in range(len(labels)):\n    lab_val = np.argmax(labels[i,:])\n    pred_val = np.argmax(pred[i,:])\n    if lab_val==pred_val:\n      accu+=1\n  return accu/len(labels)\n'

In [0]:
InputNeurons = df_train_feature.shape[1]
OutputNeurons = 7
drop_rate = 0.6  # dropout parameter, drop how many percentage
learning_rate = 0.0005
beta = 0.001 #it's used in regularization as lamda
printstep = 100
traing_epochs = 50001

# define placeholder for inputs to network
rs = tf.placeholder(tf.float32) #drop rate
xs = tf.placeholder(tf.float32, [None, InputNeurons])
ys = tf.placeholder(tf.float32, [None, OutputNeurons])

In [0]:
#define hidden layer
def dense(inputs, in_size, out_size, drop_rate, activation_function=None, Wname=None, banme=None):
    # add one more layer and return the output of this layer
    Weights = tf.Variable(tf.random_normal([in_size, out_size],stddev=0.01))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    #Wx_plus_b = tf.nn.dropout(Wx_plus_b, keep_prob)
    if activation_function is None:
        outputs = Wx_plus_b
    elif activation_function=='maxout':
        outputs = tf.contrib.layers.maxout(Wx_plus_b,out_size)
    else:
        outputs = activation_function(Wx_plus_b)
    outputs = tf.nn.dropout(x=outputs, rate=drop_rate)
    return outputs, Weights, biases

In [0]:
input_layer = tf.reshape(xs, [-1,48,48,1])
conv1 = tf.layers.conv2d(inputs=input_layer, filters=32, kernel_size=[3,3], padding='same', activation=tf.nn.relu)#filter set to be [5,5]
conv2 = tf.layers.conv2d(inputs=conv1, filters=32, kernel_size=[3,3], padding='same', activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2)
conv3 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[3,3], padding='same', activation=tf.nn.relu)
conv4 = tf.layers.conv2d(inputs=conv3, filters=64, kernel_size=[3,3], padding='same', activation=tf.nn.relu)
pool2 = tf.layers.average_pooling2d(inputs=conv4, pool_size=[2,2], strides=2)
conv5 = tf.layers.conv2d(inputs=pool2, filters=128, kernel_size=[3,3], padding='same', activation=tf.nn.relu)
conv6 = tf.layers.conv2d(inputs=conv5, filters=128, kernel_size=[3,3], padding='same', activation=tf.nn.relu)
pool3 = tf.layers.average_pooling2d(inputs=conv6, pool_size=[2,2], strides=2)
pool_flat = tf.reshape(pool3, [-1, 6*6*128])
[dense1, w1, b1] = dense(pool_flat, 6*6*128, 1024, rs, activation_function=tf.nn.relu)
[dense2, w2, b2] = dense(dense1, 1024, 512, rs, activation_function=tf.nn.relu)
logits = tf.layers.dense(inputs=dense2, units=OutputNeurons)
prediction = tf.nn.softmax(logits)

In [0]:
#calculate loss function
regularizers = tf.nn.l2_loss(w1)+tf.nn.l2_loss(w2)
#loss = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction), axis=1))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=ys, logits=logits))
loss = loss + beta*regularizers
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# start of the neural network
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

TrainLoss=[]
ValidLoss=[]
epoch=[]
for i in range(traing_epochs):
    # training
    idx = np.random.choice(np.arange(len(df_train_feature)), 64, replace=False)
    x_train_batch = df_train_feature.loc[idx,:]
    y_train_batch = arr_train_label[idx,:]
    sess.run(train_step, feed_dict={xs: x_train_batch, ys: y_train_batch, rs: drop_rate})
    if i % printstep == 0:
        # to see the step improvement
        train_loss = sess.run(loss, feed_dict={xs: x_train_batch, ys: y_train_batch, rs: 0})
        validation_loss = sess.run(loss, feed_dict={xs: df_valid_feature, ys: arr_valid_label, rs:0})
        print("epoch",i,"validation loss=", validation_loss,"Train Loss=",train_loss)
        TrainLoss.append(train_loss)
        ValidLoss.append(validation_loss)
        epoch.append(i)

epoch 0 validation loss= 2.1083019 Train Loss= 2.0704026
epoch 100 validation loss= 1.8357298 Train Loss= 1.9440422


KeyboardInterrupt: ignored

In [0]:
#%% plot loss
fig = plt.figure()
# 111代表在subplot圖中的位置
ax = fig.add_subplot(111)
ax.plot(epoch[1:], TrainLoss[1:], c='b', label='TrainLoss', linewidth=0.5)
ax.plot(epoch[1:], ValidLoss[1:], c='r', label='ValidLoss', linewidth=0.5)
leg = plt.legend()
# get the lines and texts inside legend box
leg_lines = leg.get_lines()
leg_texts = leg.get_texts()
# bulk-set the properties of all lines and texts
plt.setp(leg_lines, linewidth=6)
plt.setp(leg_texts, fontsize='x-large')
plt.show()

In [0]:
# Save Model
saver = tf.train.Saver()
save_path = saver.save(sess, path+"StoredModel/v4_model.ckpt")

In [0]:
'''
#Restore Model
sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess, path+"StoredModel/v4_model.ckpt")
'''

In [0]:
'''
arr_train_pred = sess.run(prediction, feed_dict={xs: df_train_feature, ys: arr_train_label, rs: 0})
arr_valid_pred = sess.run(prediction, feed_dict={xs: df_valid_feature, ys: arr_valid_label, rs: 0})
'''

In [0]:
'''
train_accuracy = accuracy(arr_train_label, arr_train_pred)
print("train_accuracy = ", train_accuracy*100,"%")
valid_accuracy = accuracy(arr_valid_label, arr_valid_pred)
print("valid_accuracy = ", valid_accuracy*100,"%")
'''